# Exploration
This notebook is used for data exploration for this fraud detection project.

In [3]:
import numpy as np
import pandas as pd
import sqlite3 


In [4]:
# make sqlite database out of raw data
# TODO: do this for all csv files, make separate train and test databases
  
# Connect to SQLite database 
conn = sqlite3.connect(r'../data/processed_data/credit.db') 
  
# Load CSV data into Pandas DataFrame 
train_data = pd.read_csv('../data/raw_data/train_identity.csv') 
# Write the data to a sqlite table 
train_data.to_sql('identity', conn, if_exists='replace', index=False) 
  
# Create a cursor object 
cur = conn.cursor() 
# Fetch and display result 
n = 0
for row in cur.execute('SELECT * FROM identity'): 
    n += 1
    print(row) 
    if n > 20:
        break
# Close connection to SQLite database 
conn.close() 


DatabaseError: Execution failed on sql 'DROP TABLE "identity"': database is locked